# Import Packages

In [1]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchParameters, FedBatchCellLine, FedBatchExpriment, FedBatchCellCulture

import pandas as pd
pd.set_option('display.max_columns', 200)

# 1. Analyze One Experiment in a Cell line

## 1.1. Cell Culture Data Handler

### ___FedBatchCellCulture()___ Class <br>
Loading, formatting, and pre-procesing a row measured data file. <br>
Analyzing the all cell lines and experiments or paticular cell lines.

In [2]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

fed_batch_culture = FedBatchCellCulture()

#### ___load_data()___ <br>
Loading a measured data file.
##### Parameters
----------------
    file : str
        file path of a measured data file.


In [3]:
path = input_path('fed_batch_data.xlsx')
fed_batch_culture.load_data(file=path)

#### ___get_pre_process_data()___ <br>
Get dictionary of pre-processed data to be used in cell line data handler

In [4]:
cell_culture_data = fed_batch_culture.get_pre_process_data()
cell_culture_data.keys()

dict_keys(['spc_conc_before_feed', 'spc_conc_after_feed', 'spc_feed_conc', 'spc_measured_cumulative_conc', 'exp_data', 'feed_volume', 'conc_before_feed', 'conc_after_feed', 'measured_cumulative_conc', 'feed_conc', 'polynomial_degree_data'])

## 1.2 Cell Line Data Handler Class

### ___FedBatchCellLine()___ Class <br>
Formatting the pre-processed data from cell culture handler for a particular cell line, and <br>
analyzing the cell line data.

#### Parameters
---------------
    cell_line_name : str
        Cell Line name to be analyzed.
    data : dictionary
        Pre-processed data from a cell culture data handler

In [5]:
from CCDPApy import FedBatchCellLine

fed_batch_cl = FedBatchCellLine(cell_line_name='CL1', 
                                data=cell_culture_data)

#### ___get_pre_process_data()___ <br>
Get dictionary pre-processed data for a particular cell line.

In [6]:
cell_line_data = fed_batch_cl.get_pre_process_data()
cell_line_data.keys()

dict_keys(['spc_conc_before_feed', 'spc_conc_after_feed', 'spc_feed_conc', 'spc_measured_cumulative_conc', 'exp_data', 'conc_before_feed', 'conc_after_feed', 'measured_cumulative_conc', 'feed_conc', 'feed_volume', 'polynomial_degree_data'])

## 1.3 Experiment Data Handler

### ___FedBatchExpriment()___ Class <br>
Formatting the pre-processed data from cell line handler for a particular experiment, and <br>
analyzing the data. <br>
#### Parameters
--------------
    cell_line_name : str.
        Cell line name of the experiment to be analyzed.
    cell_line_id : str.
        Cell line ID to process the all dataset of the cell line.
    data: dictionary.
        dictionary of the measured data from FedBatchCellLine class.
    use_feed_conc : bool.
        True if feed concentration is needed to calculate cumulative concentration. If this is True, use_conc_after_feed has to be False.
    use_conc_after_feed : bool
        True if concentration after feeding is needed to calculate cumulative concentration. If this is True, use_feed_conc has to be False.

In [7]:
from CCDPApy import FedBatchExpriment

fed_batch_exp = FedBatchExpriment(cell_line_name='CL1',
                                  cell_line_id='11',
                                  data=cell_line_data,
                                  use_feed_conc=True,
                                  use_conc_after_feed=False)

## 1.4. In-Processing

### ___in_process()___ <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites, using two-point calculation. <br>

In [8]:
fed_batch_exp.in_process()

## 1.5 Post-Processing

### 1.5.1 Polynomial Regression

#### ___polynomial()___ method <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites,using polynomial regression.

In [9]:
fed_batch_exp.polynomial()

### 1.5.2. Rolling Window Polynomial Regression

#### ___rolling_window_polynomial()___ <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites,using polynomial regression. <br>
##### Parameters
--------------
    degree : int, default=3
        Polynomial degree for a regression.
    window : int, default=6
        Window size of a regression.

In [10]:
fed_batch_exp.rolling_window_polynomial(degree=3, window=6)

## 1.6. Get Processed Data

### 1.6.1 Processed data for Cell

Cell Concentration (VCD, TCD, DCD, Viability)

In [11]:
fed_batch_exp.cell_conc.head()

,Run Time (day),Run Time (hr),value,unit,state,ID
0,0,0.000000,0.246,(10^6 cells/mL),VCD,11
1,0,23.883333,0.542,(10^6 cells/mL),VCD,11
2,1,38.550000,0.94,(10^6 cells/mL),VCD,11
3,1,47.583333,1.43,(10^6 cells/mL),VCD,11
4,2,62.650000,3.11,(10^6 cells/mL),VCD,11


Cumulative Cell Production

In [12]:
fed_batch_exp.cumulative_cell_production

,Run Time (day),Run Time (hr),value,unit,method,ID
0,0,0.000000,0.000000,(10^6 cells),twoPoint,11
1,0,23.883333,21.608000,(10^6 cells),twoPoint,11
2,1,38.550000,50.065000,(10^6 cells),twoPoint,11
3,1,47.583333,84.365000,(10^6 cells),twoPoint,11
4,2,62.650000,199.445000,(10^6 cells),twoPoint,11
5,2,71.750000,284.870000,(10^6 cells),twoPoint,11
6,3,95.533333,638.144250,(10^6 cells),twoPoint,11
7,5,121.100000,1112.506463,(10^6 cells),twoPoint,11
8,5,143.833333,1301.258082,(10^6 cells),twoPoint,11
9,6,167.400000,1430.489506,(10^6 cells),twoPoint,11


Integral of Viable Cell Concenration (IVCC)

In [13]:
fed_batch_exp.integral_viable_cell_conc

,Run Time (day),Run Time (hr),value,unit,ID
0,0,0.000000,0.000000,(10^6 cells hr/mL),11
1,0,23.883333,9.410033,(10^6 cells hr/mL),11
2,1,38.550000,20.278033,(10^6 cells hr/mL),11
3,1,47.583333,30.982533,(10^6 cells hr/mL),11
4,2,62.650000,65.183867,(10^6 cells hr/mL),11
5,2,71.750000,99.286117,(10^6 cells hr/mL),11
6,3,95.533333,264.877575,(10^6 cells hr/mL),11
7,5,121.100000,593.920575,(10^6 cells hr/mL),11
8,5,143.833333,991.753908,(10^6 cells hr/mL),11
9,6,167.400000,1452.482242,(10^6 cells hr/mL),11


Cell Growth Rate

In [14]:
fed_batch_exp.growth_rate

,Run Time (day),Run Time (hr),value,unit,method,ID
0,0,0.000000,NaN,(hr^-1),twoPoint,11
1,0,23.883333,0.031456,(hr^-1),twoPoint,11
2,1,38.550000,0.036621,(hr^-1),twoPoint,11
3,1,47.583333,0.045775,(hr^-1),twoPoint,11
4,2,62.650000,0.049121,(hr^-1),twoPoint,11
5,2,71.750000,0.037388,(hr^-1),twoPoint,11
6,3,95.533333,0.032023,(hr^-1),twoPoint,11
7,5,121.100000,0.021490,(hr^-1),twoPoint,11
8,5,143.833333,0.008038,(hr^-1),twoPoint,11
9,6,167.400000,0.004712,(hr^-1),twoPoint,11


### 1.6.2 Processed data for IgG(Product) and Metabolite

Concentration

In [15]:
fed_batch_exp.conc

,Run Time (day),Run Time (hr),value,unit,species,ID
0,0,0.000000,0,(mg/L),Igg,11
1,0,23.883333,1.083781,(mg/L),Igg,11
2,1,38.550000,1.770714,(mg/L),Igg,11
3,1,47.583333,2.715686,(mg/L),Igg,11
4,2,62.650000,4.354812,(mg/L),Igg,11
...,...,...,...,...,...,...
632,8,192.000000,0.0,(mM),Ethanolamine,11
633,8,215.650000,0,(mM),Ethanolamine,11
634,8,215.650000,0.0,(mM),Ethanolamine,11
635,9,233.466667,0,(mM),Ethanolamine,11


Cumulative Consumption/Production

In [16]:
fed_batch_exp.cumulative_conc

,Run Time (day),Run Time (hr),value,unit,state,method,species,ID,degree
0,0,0.000000,0.000000,(mg),Produced,twoPoint,Igg,11,NaN
1,0,23.883333,0.079116,(mg),Produced,twoPoint,Igg,11,NaN
2,1,38.550000,0.128232,(mg),Produced,twoPoint,Igg,11,NaN
3,1,47.583333,0.194380,(mg),Produced,twoPoint,Igg,11,NaN
4,2,62.650000,0.306660,(mg),Produced,twoPoint,Igg,11,NaN
...,...,...,...,...,...,...,...,...,...
645,5,143.833333,0.000000,(mmol),Consumed,polynomial,Ethanolamine,11,3.0
646,6,167.400000,0.000000,(mmol),Consumed,polynomial,Ethanolamine,11,3.0
647,8,192.000000,0.000000,(mmol),Consumed,polynomial,Ethanolamine,11,3.0
648,8,215.650000,0.000000,(mmol),Consumed,polynomial,Ethanolamine,11,3.0


SP. Rate

In [17]:
fed_batch_exp.sp_rate

,Run Time (day),Run Time (hr),value,unit,method,species,ID,degree,window
0,0.0,0.000000,NaN,(mg/10^9 cells/hr),twoPoint,Igg,11,NaN,NaN
1,0.0,23.883333,0.115173,(mg/10^9 cells/hr),twoPoint,Igg,11,NaN,NaN
2,1.0,38.550000,0.063207,(mg/10^9 cells/hr),twoPoint,Igg,11,NaN,NaN
3,1.0,47.583333,0.088278,(mg/10^9 cells/hr),twoPoint,Igg,11,NaN,NaN
4,2.0,62.650000,0.047926,(mg/10^9 cells/hr),twoPoint,Igg,11,NaN,NaN
...,...,...,...,...,...,...,...,...,...
945,5.0,132.466667,0.0,(mmol/10^9 cells/hr),rollingWindowPolynomial,Ethanolamine,11,3.0,6.0
946,5.5,155.616667,0.0,(mmol/10^9 cells/hr),rollingWindowPolynomial,Ethanolamine,11,3.0,6.0
947,7.0,179.700000,0.0,(mmol/10^9 cells/hr),rollingWindowPolynomial,Ethanolamine,11,3.0,6.0
948,8.0,203.825000,0.0,(mmol/10^9 cells/hr),rollingWindowPolynomial,Ethanolamine,11,3.0,6.0


### 1.6.3 Processed data for Each Species

#### ___get_species()___ <br>
##### Paramters
---------------
    species : str, optional, default='all'.
        Species name. 
        If 'all' is passed, the dictionary of all speces data it to be returned.

##### Cell

In [18]:
cell = fed_batch_exp.get_species('cell')

In [19]:
cell.growth_rate.head()

,Run Time (day),Run Time (hr),value,unit,method,ID
0,0.0,0.000000,NaN,(hr^-1),twoPoint,11
1,0.0,23.883333,0.031456,(hr^-1),twoPoint,11
2,1.0,38.550000,0.036621,(hr^-1),twoPoint,11
3,1.0,47.583333,0.045775,(hr^-1),twoPoint,11
4,2.0,62.650000,0.049121,(hr^-1),twoPoint,11


##### Glucose

In [51]:
glc = fed_batch_exp.get_species('glucose')
glc.sp_rate_rolling

,Run Time (day),Run Time (hr),value,unit,method,degree,window
0,0.0,11.941667,0.183481,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
1,0.5,31.216667,0.207717,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
2,1.0,43.066667,0.173014,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
3,1.5,55.116667,0.121334,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
4,2.0,67.200000,0.065351,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
5,2.5,83.641667,0.049146,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
6,4.0,108.316667,0.036860,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
7,5.0,132.466667,0.033581,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
8,5.5,155.616667,0.034065,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6
9,7.0,179.700000,0.034420,(mmol/10^9 cells/hr),rollingWindowPolynomial,3,6


### 1.6.4 Processed data of Each Profile

#### ___get_measured_data()___

In [21]:
fed_batch_exp.get_measured_data().head()

,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Run Time (day),Run Time (hr)
0,2019-08-16 17:29:00,CL1,11,XXX XXX,74,0,1,74.0,73.0,0,0,0.246,0,0.246,100.0,0,0,0.000000
1,2019-08-17 17:22:00,CL1,11,XXX XXX,74,1,1.5,73.0,71.5,0,0,0.542,0.005475,0.547475,99.0,1.083781,0,23.883333
2,2019-08-18 08:02:00,CL1,11,XXX XXX,74,2,1.5,71.5,70.0,0,0,0.94,0.005674,0.945674,99.4,1.770714,1,38.550000
3,2019-08-18 17:04:00,CL1,11,XXX XXX,74,3,1.5,70.0,68.5,0,0,1.43,0.011532,1.441532,99.2,2.715686,1,47.583333
4,2019-08-19 08:08:00,CL1,11,XXX XXX,74,4,1.5,68.5,67.0,0,0,3.11,0.076475,3.186475,97.6,4.354812,2,62.650000


#### ___get_conc_before_feed()___

In [22]:
fed_batch_exp.get_conc_before_feed().head()

,Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM)
0,0.076462,2.711509,3.655453,1.509212,0.589222,27.502389,2.371408,0.794357,2.332493,0.884815,1.606399,0,3.395402,2.922541,0.976727,1.594169,0.737122,2.510111,4.620325,3.153947,0.387326,1.49064,1.834439,0
1,0.195954,2.728914,3.693778,1.577129,0.553145,25.426547,1.972562,0.90152,2.390347,0.864469,1.611343,2.930727,3.379283,2.94189,0.968732,2.101163,0.742478,2.51341,4.596408,3.222477,0.383284,1.539198,1.845948,0
2,0.312524,2.749144,3.703455,1.63662,0.52487,23.974881,1.594671,1.047755,2.478781,0.824694,1.618828,6.890056,3.375328,2.922704,0.9589,2.560886,0.734706,2.517391,4.52893,3.285666,0.376588,1.49178,1.848962,0
3,0.428486,2.731908,3.681818,1.668991,0.503864,21.466013,1.275137,1.157412,2.537515,0.807907,1.609322,8.936312,3.350165,2.929565,0.963106,3.193406,0.731026,2.513687,4.443186,3.325389,0.379367,1.532947,1.839888,0
4,0.704902,2.698144,3.635864,1.737146,0.472736,17.376942,0.703307,1.338817,2.633449,0.748843,1.615085,16.143502,3.295428,2.90085,0.934958,4.123119,0.70738,2.512063,4.210601,3.310526,0.373872,1.500046,1.815852,0


#### ___get_conc_after_feed()___

In [23]:
fed_batch_exp.get_conc_after_feed().head()

,Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.195954,2.728914,3.693778,1.577129,0.553145,25.426547,1.972562,0.901520,2.390347,0.864469,1.611343,2.930727,3.379283,2.941890,0.968732,2.101163,0.742478,2.513410,4.596408,3.222477,0.383284,1.539198,1.845948,0.0
2,0.312524,2.749144,3.703455,1.636620,0.524870,23.974881,1.594671,1.047755,2.478781,0.824694,1.618828,6.890056,3.375328,2.922704,0.958900,2.560886,0.734706,2.517391,4.528930,3.285666,0.376588,1.491780,1.848962,0.0
3,0.428486,2.731908,3.681818,1.668991,0.503864,21.466013,1.275137,1.157412,2.537515,0.807907,1.609322,8.936312,3.350165,2.929565,0.963106,3.193406,0.731026,2.513687,4.443186,3.325389,0.379367,1.532947,1.839888,0.0
4,0.704902,2.698144,3.635864,1.737146,0.472736,17.376942,0.703307,1.338817,2.633449,0.748843,1.615085,16.143502,3.295428,2.900850,0.934958,4.123119,0.707380,2.512063,4.210601,3.310526,0.373872,1.500046,1.815852,0.0


#### ___get_cumulative_conc()___

In [24]:
fed_batch_exp.get_cumulative_conc().head()

,Cell (10^6 cells),IVCC (10^6 cells hr/mL),Igg (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mmol),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol)
0,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,21.608,9.410033,0.079116,-0.008723,-0.001271,-0.002798,-0.004958,0.002634,0.151536,0.029116,-0.007823,-0.004223,0.001485,-0.000361,0.213943,0.001177,-0.001412,0.000584,0.037011,-0.000391,-0.000241,0.001746,-0.005003,0.000295,-0.003545,-0.000840,0.0
2,50.065,20.278033,0.128232,-0.017058,-0.002717,-0.003490,-0.009212,0.004655,0.255331,0.056135,-0.018279,-0.010546,0.004329,-0.000896,0.497035,0.001459,-0.000041,0.001287,0.069881,0.000165,-0.000525,0.006571,-0.009521,0.000774,-0.000154,-0.001056,0.0
3,84.365,30.982533,0.194380,-0.025175,-0.001511,-0.001975,-0.011478,0.006126,0.430951,0.078502,-0.025955,-0.014658,0.005504,-0.000231,0.640273,0.003221,-0.000521,0.000992,0.114157,0.000422,-0.000266,0.012573,-0.012301,0.000579,-0.003036,-0.000420,0.0
4,199.445,65.183867,0.306660,-0.044110,0.000802,0.001173,-0.016146,0.008258,0.711053,0.117673,-0.038381,-0.021229,0.009550,-0.000625,1.133966,0.006970,0.001446,0.002920,0.177843,0.002042,-0.000155,0.028505,-0.011283,0.000956,-0.000782,0.001226,0.0


#### ___get_sp_rate()___

In [25]:
fed_batch_exp.get_sp_rate().head()

,Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.031456,0.115173,-0.012698,-0.001850,-0.004073,-0.007218,0.003834,0.220599,0.042385,-0.011388,-0.006148,0.002162,-0.000525,0.311447,0.001713,-0.002056,0.000850,0.053878,-0.000569,-0.000351,0.002542,-0.007283,0.000430,-0.005160,-0.001223,0.0
2,0.036621,0.063207,-0.010726,-0.001861,-0.000890,-0.005474,0.002602,0.133573,0.034771,-0.013456,-0.008137,0.003660,-0.000689,0.364311,0.000364,0.001765,0.000905,0.042301,0.000715,-0.000366,0.006209,-0.005814,0.000616,0.004363,-0.000277,0.0
3,0.045775,0.088278,-0.010833,0.001610,0.002021,-0.003024,0.001962,0.234375,0.029851,-0.010244,-0.005487,0.001568,0.000888,0.191158,0.002351,-0.000641,-0.000393,0.059089,0.000344,0.000346,0.008010,-0.003711,-0.000260,-0.003846,0.000848,0.0
4,0.049121,0.047926,-0.008082,0.000987,0.001344,-0.001993,0.000910,0.119559,0.016720,-0.005304,-0.002805,0.001727,-0.000168,0.210728,0.001600,0.000840,0.000823,0.027184,0.000691,0.000047,0.006800,0.000435,0.000161,0.000962,0.000703,0.0


#### ___get_sp_rate_poly()___

In [26]:
fed_batch_exp.get_sp_rate_poly().head()

,Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.037061,-0.081501,0.005516,-0.003009,-0.01215,-0.011646,0.001753,0.220353,0.048009,-0.021631,-0.012537,0.004759,-0.005465,0.51787,-0.0071,-0.003202,-0.000951,0.069148,-0.000835,-0.003404,0.010259,-0.009346,-0.000628,-0.002156,-0.003588,0.0
2,0.084404,0.160087,-0.022937,0.001957,0.01016,-0.000429,0.003354,0.168203,0.032327,-0.008788,-0.00425,0.002287,0.001257,0.367818,0.004371,0.003408,0.001646,0.045551,0.001994,0.001722,0.013428,-0.002063,0.000697,0.000851,0.002403,0.0
3,0.086891,0.188157,-0.024617,0.002698,0.012845,0.00211,0.003108,0.13366,0.023174,-0.003771,-0.001675,0.001326,0.002531,0.236875,0.006148,0.004255,0.001927,0.032191,0.00227,0.00258,0.011721,-0.000056,0.000868,0.001352,0.00333,0.0
4,0.060681,0.147349,-0.017505,0.002228,0.009894,0.002649,0.002048,0.081834,0.011871,0.000171,0.000043,0.000462,0.002382,0.086398,0.00515,0.003363,0.001477,0.01607,0.00172,0.002235,0.007349,0.000941,0.000689,0.00117,0.00279,0.0


#### ___get_sp_rate_rolling()___

In [27]:
fed_batch_exp.get_sp_rate_rolling().head()

,Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr)
0,0.0,11.941667,0.043802,0.123387,-0.012017,-0.003034,-0.004131,-0.007189,0.003811,0.183481,0.041186,-0.011745,-0.006639,0.002329,-0.000739,0.300984,0.000848,-0.001418,0.000757,0.047781,-0.000500,-0.000494,0.001231,-0.009075,0.000591,-0.003491,-0.001409,0.0
1,0.5,31.216667,0.042749,0.025247,-0.010518,0.000514,-0.001171,-0.005655,0.002793,0.207717,0.038171,-0.013996,-0.007998,0.003247,0.000051,0.358298,0.001823,0.000174,0.000614,0.058309,0.000391,-0.000024,0.008033,-0.003195,0.000153,0.000664,-0.000093,0.0
2,1.0,43.066667,0.041543,0.057659,-0.010243,0.000679,0.000938,-0.003589,0.001819,0.173014,0.027997,-0.010030,-0.005640,0.002503,0.000029,0.278983,0.001703,0.000746,0.000569,0.045170,0.000671,0.000045,0.007990,-0.002173,0.000151,0.000791,0.000534,0.0
3,1.5,55.116667,0.039606,0.072076,-0.007988,0.000164,0.001255,-0.002057,0.000641,0.121334,0.015634,-0.005529,-0.002895,0.001473,-0.000395,0.178459,0.000785,0.000411,0.000315,0.027108,0.000487,-0.000288,0.005579,-0.001658,0.000071,-0.000041,0.000514,0.0
4,2.0,67.200000,0.036669,0.093373,-0.006717,0.001579,0.004551,0.001035,0.001396,0.065351,0.011705,-0.001363,-0.000711,0.000638,0.001214,0.064299,0.003000,0.002297,0.000994,0.018781,0.001304,0.001381,0.006514,0.000101,0.000431,0.000787,0.001849,0.0


#### ___get_processed_data()___

In [28]:
fed_batch_exp.get_processed_data().head()

,Experiment Data,,,,,,,,,,,,,,,,,,Concentration Before Feeding,,,,,,,,,,,,,,,,,,,,,,,,Concentration After Feeding,,,,,,,,,,,,,,,,,,,,,,,,Cumulative Consumption/Production,,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Two-Point Calculation,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Rolling Window Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Run Time (day),Run Time (hr),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Cell (10^6 cells),IVCC (10^6 cells hr/mL),Igg (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mmol),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cel

## 1.7. Code

In [59]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

# Parameters
cl_name, cl_id = 'CL2', '21'
use_feed_conc, use_conc_after_feed = False, True

# Cell culture data handler
fed_batch_culture = FedBatchCellCulture()
fed_batch_culture.load_data(file=input_path('fed_batch_data.xlsx'))
cell_culture_data = fed_batch_culture.get_pre_process_data()

# Cell line data handler
cell_line_handler = FedBatchCellLine(
    cell_line_name=cl_name, 
    data=cell_culture_data)
cell_line_data = cell_line_handler.get_pre_process_data()

# Eexperiment data handler
exp_handler = FedBatchExpriment(
    cell_line_name=cl_name, 
    cell_line_id=cl_id, 
    data=cell_line_data,
    use_feed_conc=use_feed_conc,
    use_conc_after_feed=use_conc_after_feed)

# Data processing
exp_handler.in_process()
exp_handler.polynomial()
exp_handler.rolling_window_polynomial(degree=3, window=6)

# Species
spc = exp_handler.get_species('alanine')
a = spc.sp_rate_rolling.copy()

spc = exp_handler.get_species('glucose')
b = spc.sp_rate_rolling.copy()

# 2. Analyze All Experiments in a Cell Line

## 2.1 Cell Culture Data Handler

In [29]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

fed_batch_culture = FedBatchCellCulture()

# Load a data file
path = input_path('fed_batch_data.xlsx')
fed_batch_culture.load_data(file=path)

# Get pre-processed data
cell_culture_data = fed_batch_culture.get_pre_process_data()

Cell line names

In [30]:
fed_batch_culture.get_cell_line_names()

['CL1', 'CL2', 'CL3']

## 2.2 Cell Line Data Handler

In [31]:
from CCDPApy import FedBatchCellLine

cell_line = FedBatchCellLine(cell_line_name='CL1', 
                                data=cell_culture_data)

Expriment names/IDs

In [32]:
cell_line.get_experiment_names()

['11', '12', '13']

## 2.1 In-Processing

### ___in_process()___ <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites, using two-point calculation. <br>
#### Parameters
----------
    use_feed_conc : bool.
        True if feed concentration is needed to calculate cumulative concentration. If this is True, use_conc_after_feed has to be False.
    use_conc_after_feed : bool
        True if concentration after feeding is needed to calculate cumulative concentration. If this is True, use_feed_conc has to be False.

In [33]:
cell_line.in_process(use_feed_conc=True, use_conc_after_feed=False)

## 2.2 Post-Processing

### 2.2.1 Polynomial Regression

#### ___polynomial()___ <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites,using polynomial regression.

In [34]:
cell_line.polynomial()

### 2.2.2 Rolling Polynomial Regression

#### ___rolling_window_polynomial()___ <br>
Calculate cumulative concentrations and specific rates of the cell, IgG and metabolites,using polynomial regression. <br>
##### Parameters
--------------
    degree : int
        Polynomial degree for a regression.
    window : int
        Window size of a regression.

In [35]:
cell_line.rolling_window_polynomial(deg=3, window=6)

## 2.3 Get Processed Data

### ___get_processed_data()___

In [36]:
cell_line.get_processed_data().head()

,Experiment Data,,,,,,,,,,,,,,,,,,Concentration Before Feeding,,,,,,,,,,,,,,,,,,,,,,,,Concentration After Feeding,,,,,,,,,,,,,,,,,,,,,,,,Cumulative Consumption/Production,,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Two-Point Calculation,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Rolling Window Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Run Time (day),Run Time (hr),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Cell (10^6 cells),IVCC (10^6 cells hr/mL),Igg (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mmol),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cel

# 3. Analyze All Cell Lines and Experiments in Cell Culture

## 3.1 Cell Culture Data Handler

In [ ]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

fed_batch_culture = FedBatchCellCulture()

# Load a data file
path = input_path('fed_batch_data.xlsx')
fed_batch_culture.load_data(file=path)

# Get pre-processed data
cell_culture_data = fed_batch_culture.get_pre_process_data()

## 3.2 Define Parameters for Data Analysis

### ___FedBatchParameters()___ <br>
Define the parameteres for fed-batch cell culture data processing for each cell line.
#### Parameters
---------------
    cell_line_name : str
        Cell line name
    use_feed_concentration : bool
        True if feed concentration is used for the calculation of cumulative concentraion.
    use_concentration_after_feed : bool
        True if concentration after feeding is used for the calculation of cumulative concentraion.
    regression_method : list of str, default=[], optional
        Regression method. Choose 'polynomial', 'rolling_window_polynomial'.
    rolling_polynomial_degree : int, default=3, optional
        polynomial degree for rolling window regression.
    rolling_polynomial_window : int, default=6, optional
        window size for rolling window regression.

In [37]:
from CCDPApy import FedBatchParameters

param = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False, use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3, rolling_polynomial_window=6
                             )
param

Cell Line: CL1
Feed concentration will be used: True
Concentration after feeding will be used: False
Regression Methods
     Polynomial: True
     Rolling window polynomial True

## 3.3 Perform Data Processing

### ___perform_data_process()___ <br>
Perform data processing using parameters defined in FedBatchParameters class. <br>
#### Parameters
---------------
    parameters : class or list of class
        FedBatchParameters class.

In [38]:
fed_batch_culture.perform_data_process(parameters=param)

## 3.4 Get Processed Data

### ___get_processed_data()___

In [39]:
fed_batch_culture.get_processed_data().head()

,Experiment Data,,,,,,,,,,,,,,,,,,Concentration Before Feeding,,,,,,,,,,,,,,,,,,,,,,,,Concentration After Feeding,,,,,,,,,,,,,,,,,,,,,,,,Cumulative Consumption/Production,,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Two-Point Calculation,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Rolling Window Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Run Time (day),Run Time (hr),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Cell (10^6 cells),IVCC (10^6 cells hr/mL),Igg (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mmol),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cel

## 3.5 Interactive Plot

### ___interactive_plot()___

In [40]:
fed_batch_culture.interactive_plot()

Dash is running on http://127.0.0.1:8050/



## 3.6 Save the Processed Data as an Excel

### ___save_excel()___ <br>
Save the processed data as an excel file. <br>
#### Parameters
---------------
    file_name : str
        a output file name. Extention must be ".xlsx".

In [41]:
fed_batch_culture.save_excel(file_name='output_CL1.xlsx')

output_CL1.xlsx  saved.


## 3.7 Import Processed Data

In [42]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

fed_batch_culture_2 = FedBatchCellCulture()

path = input_path('fed_batch_data.xlsx')
fed_batch_culture_2.load_data(file=path)

fed_batch_culture_2.import_data(file_name='output_CL1.xlsx')

In [43]:
from CCDPApy import FedBatchParameters

param_2 = FedBatchParameters(cell_line_name='CL2',
                             use_concentration_after_feed=True, use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3, rolling_polynomial_window=6
                             )
param_2

Cell Line: CL2
Feed concentration will be used: False
Concentration after feeding will be used: True
Regression Methods
     Polynomial: True
     Rolling window polynomial True

In [44]:
fed_batch_culture_2.perform_data_process(parameters=param_2)

In [45]:
fed_batch_culture_2.get_processed_data()

,Experiment Data,,,,,,,,,,,,,,,,,,Concentration Before Feeding,,,,,,,,,,,,,,,,,,,,,,,,Concentration After Feeding,,,,,,,,,,,,,,,,,,,,,,,,Cumulative Consumption/Production,,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Two-Point Calculation,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,SP. Rate Rolling Window Polynomial Regression,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Date (MM/DD/YY H:MM:SS AM/PM),Cell Line,ID,Name,Initial Volume (mL),Sample #,Sample Volume (mL),Volume Before Sampling (mL),Volume After Sampling (mL),Feed Media Added (mL),Base Added (mL),Viable Cell Concentration (10^6 cells/mL),Dead Cell Concentration (10^6 cells/mL),Total Cell Concentration (10^6 cells/mL),Viability (%),IgG (mg/L),Run Time (day),Run Time (hr),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Alanine (mM),Arginine (mM),Asparagine (mM),Aspartate (mM),Cystine (mM),Glucose (mM),Glutamine (mM),Glutamate (mM),Glycine (mM),Histidine (mM),Isoleucine (mM),Lactate (mM),Leucine (mM),Lysine (mM),Methionine (mM),NH3 (mM),Phenylalanine (mM),Proline (mM),Serine (mM),Threonine (mM),Tryptophan (mM),Tyrosine (mM),Valine (mM),Ethanolamine (mM),Cell (10^6 cells),IVCC (10^6 cells hr/mL),Igg (mg),Alanine (mmol),Arginine (mmol),Asparagine (mmol),Aspartate (mmol),Cystine (mmol),Glucose (mmol),Glutamine (mmol),Glutamate (mmol),Glycine (mmol),Histidine (mmol),Isoleucine (mmol),Lactate (mmol),Leucine (mmol),Lysine (mmol),Methionine (mmol),Nh3 (mmol),Phenylalanine (mmol),Proline (mmol),Serine (mmol),Threonine (mmol),Tryptophan (mmol),Tyrosine (mmol),Valine (mmol),Ethanolamine (mmol),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Cell (hr^-1),Igg (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cells/hr),Serine (mmol/10^9 cells/hr),Threonine (mmol/10^9 cells/hr),Tryptophan (mmol/10^9 cells/hr),Tyrosine (mmol/10^9 cells/hr),Valine (mmol/10^9 cells/hr),Ethanolamine (mmol/10^9 cells/hr),Run Time Mid (day),Run Time Mid (hr),Cell (hr^-1),Product (mg/10^9 cells/hr),Alanine (mmol/10^9 cells/hr),Arginine (mmol/10^9 cells/hr),Asparagine (mmol/10^9 cells/hr),Aspartate (mmol/10^9 cells/hr),Cystine (mmol/10^9 cells/hr),Glucose (mmol/10^9 cells/hr),Glutamine (mmol/10^9 cells/hr),Glutamate (mmol/10^9 cells/hr),Glycine (mmol/10^9 cells/hr),Histidine (mmol/10^9 cells/hr),Isoleucine (mmol/10^9 cells/hr),Lactate (mmol/10^9 cells/hr),Leucine (mmol/10^9 cells/hr),Lysine (mmol/10^9 cells/hr),Methionine (mmol/10^9 cells/hr),Nh3 (mmol/10^9 cells/hr),Phenylalanine (mmol/10^9 cells/hr),Proline (mmol/10^9 cel

# 4. Case Study: Three Industrial Cell Lines

## 4.1 Parameter Setting

### 4.1.1 Cell line 1 <br>
Cell Line Name is "CL1". <br>
Cell Line 1 has __3 differnet experiments__.<br>
Cell Line 1 has __feed concentrations__ but no measurements of __concentrations after feeding__. <br>

In [46]:
from CCDPApy import FedBatchParameters

param_1 = FedBatchParameters(cell_line_name='CL1',
                             use_concentration_after_feed=False,
                             use_feed_concentration=True,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)
param_1

Cell Line: CL1
Feed concentration will be used: True
Concentration after feeding will be used: False
Regression Methods
     Polynomial: True
     Rolling window polynomial True

### 4.1.2 Cell line 2 <br>
Cell Line Name is "CL2". <br>
Cell Line 2 has __3 experiments__.<br>
Cell Line 2 has measurements of __concentrations after feeding__ but does't have __feed concentraions__.<br>

In [47]:
from CCDPApy import FedBatchParameters

param_2 = FedBatchParameters(cell_line_name='CL2',
                             use_concentration_after_feed=True,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)
param_2

Cell Line: CL2
Feed concentration will be used: False
Concentration after feeding will be used: True
Regression Methods
     Polynomial: True
     Rolling window polynomial True

### 4.1.3 Cell line 3 <br>
Cell Line Name is "CL3". <br>
Cell Line 3 has __3 experiments__.<br>
Cell Line 3 doesn't have both measurements of __concentrations after feeding__ and __feed concentraions__.<br>
For __glucose__, Cell Line 3 has the __calculated cumulative profile__ in Measured Data.<br>

In [48]:
from CCDPApy import FedBatchParameters

param_3 = FedBatchParameters(cell_line_name='CL3',
                             use_concentration_after_feed=False,
                             use_feed_concentration=False,
                             regression_method=['polynomial', 'rolling_window_polynomial'],
                             rolling_polynomial_degree=3,
                             rolling_polynomial_window=6)
param_3

Cell Line: CL3
Feed concentration will be used: False
Concentration after feeding will be used: False
Regression Methods
     Polynomial: True
     Rolling window polynomial True

## 4.2. Data Processing

In [49]:
from CCDPApy.helper import input_path
from CCDPApy import FedBatchCellCulture

fed_batch_culture = FedBatchCellCulture()

path = input_path('fed_batch_data.xlsx')
fed_batch_culture.load_data(file=path)
fed_batch_culture.perform_data_process(parameters=[param_1, param_2, param_3])

## 4.3. Interactive Plot

In [ ]:
fed_batch_culture.interactive_plot()